In [1]:
import glob, os
import fnmatch
import sys
import astropy
import json
import parsnip
import pandas as pd
import sncosmo
import iminuit
import csv
import lcdata
from astropy.table import QTable, Table, Column
import numpy as np
from matplotlib import pyplot as plt

In [2]:
dir = '../../../cfs/cdirs/desi/science/td/DECam/imaging/LCData_Legacy/'

In [7]:
#read from .json file
for folder in os.listdir(dir):
    if folder.startswith('T2021'):
        id = folder
        for file in os.listdir(dir+folder+'/'):
            if file.endswith('.json'):
                f = open(dir+folder + '/' + file)
                data = json.load(f)
                id = data['ObjectID']
                ra = data['RA-OBJECT']
                dec = data['DEC-OBJECT']
                type = 'Unknown'

        # read redshift of the host start from DESIRT_hosts_redshufts_fujilupe_Y1
        csv_host_rs = open('DESIRT_hosts_redshifts_fujilupe_Y1.csv')
        reader_host = csv.DictReader(csv_host_rs)
        for row in reader_host:
            if row['# TransientID'] == id:
                rs = row[' REDSHIFT']
                
        #construct a metadata table
        metaTable = Table(names=('object_id', 'ra', 'dec', 'type', 'redshift'), dtype=('S2', 'i', 'i', 'S2', 'f'))
        metaTable.add_row((id, ra, dec, type, rs))
                
        #read from .csv file
        lightcurveTable = Table(names=('time(mjd)', 'flux', 'fluxerr', 'band', 'zp', 'zpsys'), dtype=('f', 'f', 'f', 'S', 'f', 'S'))

        for file in os.listdir(dir+id+"/"):
            if file.endswith(".csv"):
                csvFile = open(dir+id+'/'+file)
                reader = csv.DictReader(csvFile)
                for row in reader:
                    if (float(row['mag']) < float(row['lm5'])) & (float(row['mag']) > 12):
                        time = row['mjd']
                        mag = float(row['mag'])
                        magerr = float(row['magerr'])
                        flux = 10**((-mag)/2.5)
                        fluxerr = fluxerr = flux * np.log(10) / 2.5 * magerr
                        band = 'des' + row['filter']
                        zp = 22.5
                        zpsys = 'ab'
                        lightcurveTable.add_row((time, flux, fluxerr, band, zp, zpsys))
                                
        #combine the two tables into lcdata
        light_curve_curr = Table({
                'time': list(lightcurveTable['time(mjd)']),
                'flux': list(lightcurveTable['flux']),
                'fluxerr': list(lightcurveTable['fluxerr']),
                'band': list(lightcurveTable['band']),
                'zp': list(lightcurveTable['zp']),
                'zpsys': list(lightcurveTable['zpsys'])})

        light_curve_curr.meta = {
            'object_id': id,
            'ra': ra,
            'dec': dec,
            'type': type,
            'redshift': rs}
                
        z_TNS = rs
        source = parsnip.ParsnipSncosmoSource('plasticc')
        model = sncosmo.Model(source=source)
        result, fitted_model = sncosmo.fit_lc(light_curve_curr, model,['t0', 'amplitude', 's1', 's2', 's3', 'color'],bounds={'z': (z_TNS,z_TNS)})
        
        sncosmo.plot_lc(light_curve_curr, model=fitted_model, errors=result.errors)
        plt.savefig('All Light Curve Plots with des/lightcurve-'+id+'.png',dpi=200)

UFuncTypeError: ufunc 'add' did not contain a loop with signature matching types (dtype('<U32'), dtype('<U32')) -> dtype('<U32')